In [34]:
import pandas as pd
import plotly.express as px
import os
from scipy.stats import norm
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt


In [35]:
#import .csvs
reg_season_wins = pd.read_csv('nfl_reg_season_win_loss_records_2011_to_2022.csv')
playoff_wins = pd.read_csv('nfl_playoff_win_loss_records_2011_to_2022.csv')
superbowl_wins = pd.read_csv('Super_Bowl_Winners.csv')
player_salaries = pd.read_csv('nfl_salaries_2011-2022.csv')

In [36]:
reg_season_wins.head(60)

,Year,Team,Win,Loss,Tie
0,2011,Green Bay,15,1,0
1,2011,San Francisco,13,3,0
2,2011,New England,13,3,0
3,2011,New Orleans,13,3,0
4,2011,Baltimore,12,4,0
5,2011,Pittsburgh,12,4,0
6,2011,Atlanta,10,6,0
7,2011,Detroit,10,6,0
8,2011,Houston,10,6,0
9,2011,NY Giants,9,7,0


In [37]:
playoff_wins

,Year,Team,Win,Loss,Tie
0,2011,NY Giants,4,0,0
1,2011,New England,2,1,0
2,2011,Baltimore,1,1,0
3,2011,Houston,1,1,0
4,2011,Denver,1,1,0
...,...,...,...,...,...
145,2022,Minnesota,0,1,0
146,2022,Baltimore,0,1,0
147,2022,LA Chargers,0,1,0
148,2022,Seattle,0,1,0


In [38]:
superbowl_wins

,year,team,superbowl wins
0,2011,NY Giants,1
1,2012,Baltimore,1
2,2013,Seattle,1
3,2014,New England,1
4,2015,Denver,1
5,2016,New England,1
6,2017,Philadelphia,1
7,2018,New England,1
8,2019,Kansas City,1
9,2020,Tampa Bay,1


In [39]:
player_salaries

,index,year,team,name,position,cap_hit,cap_percentage
0,1,2011,arizona-cardinals,Larry Fitzgerald,WR,"$2,000,000",13.28
1,2,2011,arizona-cardinals,Levi Brown,LT,"$6,777,500",7.70
2,3,2011,arizona-cardinals,Adrian Wilson,S,"$3,500,000",6.17
3,4,2011,arizona-cardinals,Darnell Dockett,DE,"$2,350,000",4.58
4,5,2011,arizona-cardinals,Derek Anderson,QB,"$4,087,500",4.40
...,...,...,...,...,...,...,...
20663,20664,2022,washington-football-team,David Bada,DT,"$705,000",0.04
20664,20665,2022,washington-football-team,Nate Gerry,OLB,"$57,500",0.03
20665,20666,2022,washington-football-team,Jaret Patterson,RB,"$825,000",0.02
20666,20667,2022,washington-football-team,Alex Akingbulu,T,"$705,000",0.02


# First analysis: Grab the highest paid player (by percent of cap), bucket them into groups, and determine if theres correlation between the highest paid player and success

In [40]:
# Clean up the player_salaries DF

# Make cap % a float
player_salaries['cap_percentage'] = player_salaries['cap_percentage'].astype(float)

# Use groupby with idxmax to get the index of rows with the highest cap_percentage per year per team
idx = player_salaries.groupby(['year', 'team'])['cap_percentage'].idxmax()

# Filter the dataframe based on the index
highest_paid = player_salaries.loc[idx].reset_index(drop=True)

# standardize team name to just include city first name
def clean_team_name(team):
    parts = team.split('-')
    city_name = parts[:-1]
    if city_name[-1] == 'football':
        city_name = city_name[:-1]
    return ' '.join(word.title() for word in city_name)

# Apply the custom function to the 'team' column in highest_salary_df
highest_paid['team'] = highest_paid['team'].apply(clean_team_name)

# Display the new DataFrame
highest_paid.head(60)

,index,year,team,name,position,cap_hit,cap_percentage
0,1,2011,Arizona,Larry Fitzgerald,WR,"$2,000,000",13.28
1,59,2011,Atlanta,Matt Ryan,QB,"$11,250,000",11.01
2,112,2011,Baltimore,Terrell Suggs,OLB,"$3,400,000",7.94
3,165,2011,Buffalo,Ryan Fitzpatrick,QB,"$3,220,000",4.02
4,215,2011,Carolina,Jordan Gross,LT,"$6,000,000",8.12
5,280,2011,Chicago,Brian Urlacher,LB,"$8,025,000",9.23
6,338,2011,Cincinnati,Andre Smith,G,"$1,677,500",6.42
7,394,2011,Cleveland,Joe Thomas,LT,"$8,000,000",12.34
8,441,2011,Dallas,Terence Newman,CB,"$8,000,000",7.27
9,497,2011,Denver,Elvis Dumervil,OLB,"$14,000,000",10.67


In [41]:
# highest_paid
highest_paid

# Plotting a histogram
fig = px.histogram(highest_paid, x='cap_percentage', nbins=50, title='highest_paid')
fig.show()

In [9]:
#merge the reg season wins losses with the player salaries df

reg_season_wins.rename(columns={'Team': 'team', 'Year': 'year'}, inplace=True)
# reg_season_wins

# merge dataframes
salaries_with_reg_season_wins_df = pd.merge(highest_paid, reg_season_wins, on=['year', 'team'], how='inner')

salaries_with_reg_season_wins_df

,index,year,team,name,position,cap_hit,cap_percentage,Win,Loss,Tie
0,1,2011,Arizona,Larry Fitzgerald,WR,"$2,000,000",13.28,8,8,0
1,59,2011,Atlanta,Matt Ryan,QB,"$11,250,000",11.01,10,6,0
2,112,2011,Baltimore,Terrell Suggs,OLB,"$3,400,000",7.94,12,4,0
3,165,2011,Buffalo,Ryan Fitzpatrick,QB,"$3,220,000",4.02,6,10,0
4,215,2011,Carolina,Jordan Gross,LT,"$6,000,000",8.12,6,10,0
...,...,...,...,...,...,...,...,...,...,...
331,20406,2022,San Francisco,Jimmie Ward,FS,"$8,900,000",6.09,13,4,0
332,20459,2022,Seattle,Poona Ford,DE,"$7,900,000",4.64,9,8,0
333,20512,2022,Tampa Bay,Donovan Smith,LT,"$15,750,000",8.94,8,9,0
334,20565,2022,Tennessee,Derrick Henry,RB,"$4,000,000",4.44,7,10,0


In [10]:


# playoff_wins.drop(columns={'Loss', 'Tie'}, inplace=True)

playoff_wins.rename(columns={'Team': 'team', 'Year': 'year', 'Win': 'Playoff Wins'}, inplace=True)

# merge salaries_with_reg_season_wins_df with postseason wins
reg_and_postseason_wins_df = pd.merge(salaries_with_reg_season_wins_df, playoff_wins, on=['year', 'team'], how='left')

reg_and_postseason_wins_df['Playoff Wins'] = reg_and_postseason_wins_df['Playoff Wins'].fillna(0)

reg_and_postseason_wins_df



,index,year,team,name,position,cap_hit,cap_percentage,Win,Loss,Tie,Playoff Wins
0,1,2011,Arizona,Larry Fitzgerald,WR,"$2,000,000",13.28,8,8,0,0.0
1,59,2011,Atlanta,Matt Ryan,QB,"$11,250,000",11.01,10,6,0,0.0
2,112,2011,Baltimore,Terrell Suggs,OLB,"$3,400,000",7.94,12,4,0,1.0
3,165,2011,Buffalo,Ryan Fitzpatrick,QB,"$3,220,000",4.02,6,10,0,0.0
4,215,2011,Carolina,Jordan Gross,LT,"$6,000,000",8.12,6,10,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
331,20406,2022,San Francisco,Jimmie Ward,FS,"$8,900,000",6.09,13,4,0,2.0
332,20459,2022,Seattle,Poona Ford,DE,"$7,900,000",4.64,9,8,0,0.0
333,20512,2022,Tampa Bay,Donovan Smith,LT,"$15,750,000",8.94,8,9,0,0.0
334,20565,2022,Tennessee,Derrick Henry,RB,"$4,000,000",4.44,7,10,0,0.0


In [11]:
# merge reg_and_postseason_wins_df and superbowl wins
superbowl_wins.columns = ['year', 'team', 'Won Superbowl?']

# merge previous with superbowl wins
full_season_wins = pd.merge(reg_and_postseason_wins_df, superbowl_wins, on=['year', 'team'], how='left')

full_season_wins.fillna(0, inplace=True)

full_season_wins.head(60)


,index,year,team,name,position,cap_hit,cap_percentage,Win,Loss,Tie,Playoff Wins,Won Superbowl?
0,1,2011,Arizona,Larry Fitzgerald,WR,"$2,000,000",13.28,8,8,0,0.0,0.0
1,59,2011,Atlanta,Matt Ryan,QB,"$11,250,000",11.01,10,6,0,0.0,0.0
2,112,2011,Baltimore,Terrell Suggs,OLB,"$3,400,000",7.94,12,4,0,1.0,0.0
3,165,2011,Buffalo,Ryan Fitzpatrick,QB,"$3,220,000",4.02,6,10,0,0.0,0.0
4,215,2011,Carolina,Jordan Gross,LT,"$6,000,000",8.12,6,10,0,0.0,0.0
5,280,2011,Chicago,Brian Urlacher,LB,"$8,025,000",9.23,8,8,0,0.0,0.0
6,338,2011,Cincinnati,Andre Smith,G,"$1,677,500",6.42,9,7,0,0.0,0.0
7,394,2011,Cleveland,Joe Thomas,LT,"$8,000,000",12.34,4,12,0,0.0,0.0
8,441,2011,Dallas,Terence Newman,CB,"$8,000,000",7.27,8,8,0,0.0,0.0
9,497,2011,Denver,Elvis Dumervil,OLB,"$14,000,000",10.67,8,8,0,1.0,0.0


In [12]:
# plot cap percentage in relation to wins for the highest paid player on each team since 2011

fig = px.scatter(full_season_wins, x='cap_percentage', y='Win', hover_data=['name', 'team', 'year'], trendline='ols', trendline_color_override="red", title='Win vs Cap Percentage')

fig.show()


In [13]:
# plot cap percentage in relation to wins for the highest paid player on 10-win teams since 2011
playoff_teams_df = full_season_wins.loc[full_season_wins['Win'] >= 10].reset_index(drop=True)


fig = px.scatter(playoff_teams_df, x='cap_percentage', y='Playoff Wins', hover_data=['name', 'team', 'year'], title='Playoff Wins vs Cap Percentage')

fig.show()



In [14]:
data = playoff_teams_df

# Prepare hover text
hover_text = data.apply(lambda row: f"Name: {row['name']}<br>Team: {row['team']}<br>Year: {row['year']}", axis=1)

# Scatter plot
scatter = go.Scatter(
    x=data['cap_percentage'], 
    y=data['Playoff Wins'], 
    mode='markers', 
    name='Teams',
    text=hover_text,  # This sets the hover text
    hoverinfo='text'  # This tells plotly to use the 'text' field for hover
)

# Bell curve
x = np.linspace(min(data['cap_percentage']), max(data['cap_percentage']), 100)
mean = data['cap_percentage'].mean()
std_dev = data['cap_percentage'].std()
y = norm.pdf(x, mean, std_dev) * data['Playoff Wins'].max()  # Scale bell curve to max of y-values

bell_curve = go.Line(x=x, y=y, name='Bell Curve', line=dict(color='red'))

layout = go.Layout(
    title="Your Title",
    xaxis=dict(
        title="cap_percentage",
        range=[4, 18]  # Set the range for x-axis
    ),
    yaxis=dict(
        title="Win",
        tickvals=list(range(0, int(data['Playoff Wins'].max())+1))  # Set the tick values for y-axis to whole numbers
    )
)

fig = go.Figure(data=[scatter, bell_curve], layout=layout)
fig.show()

/Users/danieldreger/anaconda3/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




# Depth of talent in relation to wins

In [15]:
# Create a new dataframe with number of players over a certain % of the cap
# 2% of the cap last year was just under 5 million dollars

player_salaries.head(60)

# Filter players with cap_percentage over 2.0
players_over_2_percent = player_salaries[player_salaries['cap_percentage'] > 2.0]

# Group by year and team, then count players
players_over_2_percent_by_team = players_over_2_percent.groupby(['year', 'team']).size().reset_index(name='players_over_2%')

# Resetting the index for your desired format
players_over_2_percent_by_team.reset_index(inplace=True)
players_over_2_percent_by_team['index'] = players_over_2_percent_by_team.index + 3

players_over_2_percent_by_team






,index,year,team,players_over_2%
0,3,2011,arizona-cardinals,12
1,4,2011,atlanta-falcons,15
2,5,2011,baltimore-ravens,13
3,6,2011,buffalo-bills,9
4,7,2011,carolina-panthers,13
...,...,...,...,...
379,382,2022,san-francisco-49ers,11
380,383,2022,seattle-seahawks,9
381,384,2022,tampa-bay-buccaneers,10
382,385,2022,tennessee-titans,4


In [16]:
# players_over_2_percent_by_team

# Plotting a histogram
fig = px.histogram(players_over_2_percent_by_team, x='players_over_2%', nbins=50, title='Distribution of Salaries')
fig.show()





In [17]:
# clean data

# standardize team name to just include city first name
def clean_team_name(team):
    parts = team.split('-')
    city_name = parts[:-1]
    if city_name[-1] == 'football':
        city_name = city_name[:-1]
    return ' '.join(word.title() for word in city_name)

# Apply the custom function to the 'team' column in highest_salary_df
players_over_2_percent_by_team['team'] = players_over_2_percent_by_team['team'].apply(clean_team_name)

# Display the new DataFrame
players_over_2_percent_by_team.tail(60)


,index,year,team,players_over_2%
324,327,2021,Carolina,9
325,328,2021,Chicago,10
326,329,2021,Cincinnati,13
327,330,2021,Cleveland,10
328,331,2021,Dallas,9
329,332,2021,Denver,8
330,333,2021,Detroit,4
331,334,2021,Green Bay,12
332,335,2021,Houston,7
333,336,2021,Indianapolis,13


In [18]:
# merge dataframes

# merge salaries with reg season wins
roster_depth_with_reg_season_wins_df = pd.merge(players_over_2_percent_by_team, reg_season_wins, on=['year', 'team'], how='inner')

# merge previous with playoff wins
prev_and_postseason_wins_df = pd.merge(roster_depth_with_reg_season_wins_df, playoff_wins, on=['year', 'team'], how='left')

# merge previous with superbowl wins
roster_depth_and_full_season_wins = pd.merge(prev_and_postseason_wins_df, superbowl_wins, on=['year', 'team'], how='left')

roster_depth_and_full_season_wins[['Playoff Wins', 'Won Superbowl?']] = roster_depth_and_full_season_wins[['Playoff Wins', 'Won Superbowl?']].fillna(0)


roster_depth_and_full_season_wins



,index,year,team,players_over_2%,Win,Loss,Tie,Playoff Wins,Won Superbowl?
0,3,2011,Arizona,12,8,8,0,0.0,0.0
1,4,2011,Atlanta,15,10,6,0,0.0,0.0
2,5,2011,Baltimore,13,12,4,0,1.0,0.0
3,6,2011,Buffalo,9,6,10,0,0.0,0.0
4,7,2011,Carolina,13,6,10,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
331,382,2022,San Francisco,11,13,4,0,2.0,0.0
332,383,2022,Seattle,9,9,8,0,0.0,0.0
333,384,2022,Tampa Bay,10,8,9,0,0.0,0.0
334,385,2022,Tennessee,4,7,10,0,0.0,0.0


In [19]:
# plot roster depth in relation to wins for the highest paid player on each team since 2011

fig = px.scatter(roster_depth_and_full_season_wins, x='players_over_2%', y='Win', hover_data=['team', 'year'], trendline='ols', trendline_color_override="red", title='Wins vs Roster Depth')

fig.show()

# Analyze spending by Offense/Defense


In [20]:
# Define offensive and defensive positions
offensive_positions = ['WR', 'QB', 'LT', 'RB', 'C', 'RT', 'TE', 'G', 'FB']
defensive_positions = ['ILB', 'OLB', 'LB', 'CB', 'S', 'DE', 'DT']

# Map positions to categories
def map_to_category(pos):
    if pos in offensive_positions:
        return 'offense'
    elif pos in defensive_positions:
        return 'defense'
    else:
        return 'other'  # for any positions not listed in either list

player_salaries['category'] = player_salaries['position'].apply(map_to_category)

# Filter out rows that fall into the 'other' category (if any)
player_salaries = player_salaries[player_salaries['category'] != 'other']

# Group by year, team, and category, then sum up the cap_percentage values
grouped = player_salaries.groupby(['year', 'team', 'category'])['cap_percentage'].sum().unstack().reset_index()

# Rename the columns
grouped.columns.name = None  # remove the top-level category name
grouped = grouped.rename(columns={'offense': 'offense_spending', 'defense': 'defense_spending'})

print(grouped)

     year                      team  defense_spending  offense_spending
0    2011         arizona-cardinals             37.21             49.88
1    2011           atlanta-falcons             39.49             54.22
2    2011          baltimore-ravens             45.99             35.33
3    2011             buffalo-bills             27.49             19.62
4    2011         carolina-panthers             35.41             52.20
..    ...                       ...               ...               ...
379  2022       san-francisco-49ers             30.87             32.70
380  2022          seattle-seahawks             21.81             26.36
381  2022      tampa-bay-buccaneers             32.73             40.65
382  2022          tennessee-titans             15.33             16.62
383  2022  washington-football-team             26.84             44.89

[384 rows x 4 columns]


In [21]:
# Create histograms
trace1 = go.Histogram(
    x=grouped['offense_spending'],
    opacity=0.75,
    name='Offense Spending',
    marker=dict(color='blue')
)

trace2 = go.Histogram(
    x=grouped['defense_spending'],
    opacity=0.75,
    name='Defense Spending',
    marker=dict(color='red')
)

# Layout
layout = go.Layout(
    title='Distribution of Spending on Offense vs. Defense',
    xaxis=dict(title='Spending (%)'),
    yaxis=dict(title='Count'),
    barmode='overlay'
)

# Combine traces and layout to create a figure, then plot
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()


In [22]:
# clean data

# standardize team name to just include city first name
def clean_team_name(team):
    parts = team.split('-')
    city_name = parts[:-1]
    if city_name[-1] == 'football':
        city_name = city_name[:-1]
    return ' '.join(word.title() for word in city_name)

# Apply the custom function to the 'team' column in highest_salary_df
grouped['team'] = grouped['team'].apply(clean_team_name)

# Display the new DataFrame
grouped

,year,team,defense_spending,offense_spending
0,2011,Arizona,37.21,49.88
1,2011,Atlanta,39.49,54.22
2,2011,Baltimore,45.99,35.33
3,2011,Buffalo,27.49,19.62
4,2011,Carolina,35.41,52.20
...,...,...,...,...
379,2022,San Francisco,30.87,32.70
380,2022,Seattle,21.81,26.36
381,2022,Tampa Bay,32.73,40.65
382,2022,Tennessee,15.33,16.62


In [23]:
# clean data and merge the offense/defense spending df with the wins/playoff wins/sb wins df


# merge salaries with reg season wins
spending_with_reg_season_wins_df = pd.merge(grouped, reg_season_wins, on=['year', 'team'], how='inner')

# merge previous with playoff wins
prev_and_postseason_wins_df = pd.merge(spending_with_reg_season_wins_df, playoff_wins, on=['year', 'team'], how='left')

# merge previous with superbowl wins
spending_and_full_season_wins = pd.merge(prev_and_postseason_wins_df, superbowl_wins, on=['year', 'team'], how='left')

spending_and_full_season_wins[['Playoff Wins', 'Won Superbowl?']] = spending_and_full_season_wins[['Playoff Wins', 'Won Superbowl?']].fillna(0)

spending_and_full_season_wins


,year,team,defense_spending,offense_spending,Win,Loss,Tie,Playoff Wins,Won Superbowl?
0,2011,Arizona,37.21,49.88,8,8,0,0.0,0.0
1,2011,Atlanta,39.49,54.22,10,6,0,0.0,0.0
2,2011,Baltimore,45.99,35.33,12,4,0,1.0,0.0
3,2011,Buffalo,27.49,19.62,6,10,0,0.0,0.0
4,2011,Carolina,35.41,52.20,6,10,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
331,2022,San Francisco,30.87,32.70,13,4,0,2.0,0.0
332,2022,Seattle,21.81,26.36,9,8,0,0.0,0.0
333,2022,Tampa Bay,32.73,40.65,8,9,0,0.0,0.0
334,2022,Tennessee,15.33,16.62,7,10,0,0.0,0.0


In [24]:
# Assuming you've already imported pandas and created the spending_and_full_season_wins dataframe
df = spending_and_full_season_wins

# Create scatter plot
fig = px.scatter(df, x="offense_spending", y="Win", hover_data=["team", "year"], 
                 title="Offensive Spending vs. Wins",
                 labels={"offense_spending": "Offensive Spending (%)", "Win": "Wins"},
                 trendline='ols', 
                 trendline_color_override="red",
                )

# Show plot
fig.show()

In [25]:
df = spending_and_full_season_wins

# Create scatter plot
fig = px.scatter(df, x="defense_spending", y="Win", hover_data=["team", "year"], 
                 title="defense_spending vs. Wins",
                 labels={"defense_spending": "defense_spending (%)", "Win": "Wins"},
                 trendline='ols', 
                 trendline_color_override="red",
                )

# Show plot
fig.show()

In [26]:


playoff_teams_spending_df = spending_and_full_season_wins.loc[spending_and_full_season_wins['Playoff Wins'] >= 1].reset_index(drop=True)

df = playoff_teams_spending_df

# Create scatter plot
fig = px.scatter(df, x="offense_spending", y="Playoff Wins", hover_data=["team", "year"], 
                 title="Offensive Spending vs. Playoff Wins",
                 labels={"offense_spending": "Offensive Spending (%)", "Playoff Wins": "Playoff Wins"},
                 trendline='ols', 
                 trendline_color_override="red",
                )

# Show plot
fig.show()

In [27]:
playoff_teams_spending_df = spending_and_full_season_wins.loc[spending_and_full_season_wins['Win'] >= 10].reset_index(drop=True)

df = playoff_teams_spending_df

# Create scatter plot
fig = px.scatter(df, x="defense_spending", y="Playoff Wins", hover_data=["team", "year"], 
                 title="defense_spending vs. Playoff Wins",
                 labels={"defense_spending": "defense_spending (%)", "Playoff Wins": "Playoff Wins"},
                 trendline='ols', 
                 trendline_color_override="red",
                )

# Show plot
fig.show()

# Analyze spending by Position Group, Offense and Defense




In [28]:
# Define offensive and defensive positions

quarterbacks = ['QB']
receivers = ['TE', 'WR' ]
o_line = ['LT', 'C', 'RT', 'G', ]
running_backs = ['RB', 'FB']
d_line = ['DE', 'DT']
linebackers = ['ILB', 'OLB', 'LB']
secondary = ['CB', 'S']


# Map positions to categories
def map_to_position(pos):
    if pos in quarterbacks:
        return 'quarterbacks'
    elif pos in receivers:
        return 'receivers'
    elif pos in o_line:
        return 'o_line'
    elif pos in running_backs:
        return 'running_backs'
    elif pos in d_line:
        return 'd_line'
    elif pos in linebackers:
        return 'linebackers'
    elif pos in secondary:
        return 'secondary'
    else:
        return 'special teams'  # for any positions not listed in either list

player_salaries_by_position = player_salaries['position'].apply(map_to_position)

# Filter out rows that fall into the 'other' category (if any)
player_salaries_by_position = player_salaries[player_salaries['category'] != 'other']

# # Group by year, team, and category, then sum up the cap_percentage values
grouped_positions_numbers = player_salaries_by_position.groupby(['year', 'team', 'category'])['cap_percentage'].sum().unstack().reset_index()

# # Rename the columns
grouped_positions_numbers.columns.name = None  # remove the top-level category name
grouped_positions_numbers = grouped_positions_numbers.rename(columns={'secondary': 'secondary_spending', 'linebackers': 'linebackers_spending', 'd_line': 'd_line_spending', 'running_backs': 'running_backs_spending', 'o_line': 'o_line_spending', 'receivers': 'receivers_spending', 'quarterbacks': 'quarterbacks_spending', 'special teams': 'special_teams_spending'})

grouped_positions_numbers

,year,team,defense,offense
0,2011,arizona-cardinals,37.21,49.88
1,2011,atlanta-falcons,39.49,54.22
2,2011,baltimore-ravens,45.99,35.33
3,2011,buffalo-bills,27.49,19.62
4,2011,carolina-panthers,35.41,52.20
...,...,...,...,...
379,2022,san-francisco-49ers,30.87,32.70
380,2022,seattle-seahawks,21.81,26.36
381,2022,tampa-bay-buccaneers,32.73,40.65
382,2022,tennessee-titans,15.33,16.62


In [29]:
# clean data

# standardize team name to just include city first name
def clean_team_name(team):
    parts = team.split('-')
    city_name = parts[:-1]
    if city_name[-1] == 'football':
        city_name = city_name[:-1]
    return ' '.join(word.title() for word in city_name)

# Apply the custom function to the 'team' column in highest_salary_df
grouped_positions_numbers['team'] = grouped_positions_numbers['team'].apply(clean_team_name)

# Display the new DataFrame
grouped_positions_numbers

,year,team,defense,offense
0,2011,Arizona,37.21,49.88
1,2011,Atlanta,39.49,54.22
2,2011,Baltimore,45.99,35.33
3,2011,Buffalo,27.49,19.62
4,2011,Carolina,35.41,52.20
...,...,...,...,...
379,2022,San Francisco,30.87,32.70
380,2022,Seattle,21.81,26.36
381,2022,Tampa Bay,32.73,40.65
382,2022,Tennessee,15.33,16.62


In [30]:
# merge position spending with reg season wins
pos_spending_with_reg_season_wins_df = pd.merge(grouped_positions_numbers, reg_season_wins, on=['year', 'team'], how='inner')

# merge previous with playoff wins
prev_and_postseason_wins_df = pd.merge(pos_spending_with_reg_season_wins_df, playoff_wins, on=['year', 'team'], how='left')

# merge previous with superbowl wins
pos_spending_and_full_season_wins = pd.merge(prev_and_postseason_wins_df, superbowl_wins, on=['year', 'team'], how='left')

pos_spending_and_full_season_wins[['Playoff Wins', 'Won Superbowl?']] = spending_and_full_season_wins[['Playoff Wins', 'Won Superbowl?']].fillna(0)

pos_spending_and_full_season_wins



,year,team,defense,offense,Win,Loss,Tie,Playoff Wins,Won Superbowl?
0,2011,Arizona,37.21,49.88,8,8,0,0.0,0.0
1,2011,Atlanta,39.49,54.22,10,6,0,0.0,0.0
2,2011,Baltimore,45.99,35.33,12,4,0,1.0,0.0
3,2011,Buffalo,27.49,19.62,6,10,0,0.0,0.0
4,2011,Carolina,35.41,52.20,6,10,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
331,2022,San Francisco,30.87,32.70,13,4,0,2.0,0.0
332,2022,Seattle,21.81,26.36,9,8,0,0.0,0.0
333,2022,Tampa Bay,32.73,40.65,8,9,0,0.0,0.0
334,2022,Tennessee,15.33,16.62,7,10,0,0.0,0.0


In [31]:
df = pos_spending_and_full_season_wins

# Create a list of columns to plot
spending_columns = [
    'o_line_spending', 
    'quarterbacks_spending', 'receivers_spending', 
    'running_backs_spending'
]

# Color list to use for plots
colors = px.colors.qualitative.Plotly

fig = go.Figure()

# Add scatter plots and lines of best fit for each spending column
for idx, column in enumerate(spending_columns):
    # Scatter plot
    fig.add_trace(go.Scatter(x=df[column], y=df['Win'], mode='markers',
                             hoverinfo='text+name',
                             text=df.apply(lambda row: f"Team: {row['team']}<br>Year: {row['year']}", axis=1),
                             name=column,
                             marker_color=colors[idx]))

    # Line of best fit
    x_range = np.linspace(df[column].min(), df[column].max(), 100)
    p = np.polyfit(df[column], df['Win'], 1)
    fig.add_trace(go.Scatter(x=x_range, y=np.polyval(p, x_range), mode='lines',
                             showlegend=False, line=dict(color=colors[idx])))

fig.update_layout(title='Spending vs Wins', xaxis_title='Spending', yaxis_title='Wins')

fig.show()


KeyError: 'o_line_spending'

In [270]:
df = pos_spending_and_full_season_wins

# Create a list of columns to plot
spending_columns = [
    'd_line_spending', 'linebackers_spending', 'secondary_spending'
]

# Color list to use for plots
colors = px.colors.qualitative.Plotly

fig = go.Figure()

# Add scatter plots and lines of best fit for each spending column
for idx, column in enumerate(spending_columns):
    # Scatter plot
    fig.add_trace(go.Scatter(x=df[column], y=df['Win'], mode='markers',
                             hoverinfo='text+name',
                             text=df.apply(lambda row: f"Team: {row['team']}<br>Year: {row['year']}", axis=1),
                             name=column,
                             marker_color=colors[idx]))

    # Line of best fit
    x_range = np.linspace(df[column].min(), df[column].max(), 100)
    p = np.polyfit(df[column], df['Win'], 1)
    fig.add_trace(go.Scatter(x=x_range, y=np.polyval(p, x_range), mode='lines',
                             showlegend=False, line=dict(color=colors[idx])))

fig.update_layout(title='Spending vs Wins', xaxis_title='Spending', yaxis_title='Wins')

fig.show()